# Tutorial on how to automatically select good features to track

In [ ]:
import os
import sys
repo_root = os.path.abspath("..")
sys.path.insert(0, repo_root)

import numpy as np
import matplotlib.pyplot as plt
import pyidi

Load the video

In [ ]:
filename = '../data/data_synthetic.cih'
video = pyidi.VideoReader(filename)

### `Lucas Kanade` method as a reference

In [ ]:
tol = 1e-8
roi_size = (5,5)
idi_lk = pyidi.LucasKanade(video)
idi_lk.configure(reference_image = (0,100), resume_analysis = False, tol=tol, roi_size=roi_size)
reference_image = idi_lk._set_reference_image(video, idi_lk.reference_image)

### Generate a score image
- Initialize the Feature Selector
- Set a filter type
- Set roi_size. Either directly in set_filter or trough set_roi
- Apply filter to generate the score_image (saved in feature_selector.score_image)

All available methods are listed when initializing the FeatureSelector, unless verbose = False

In [ ]:
from pyidi.Automatic_selection.feature_selector import FeatureSelector
feature_selector = FeatureSelector(reference_image)

feature_selector.set_filter('ST', roi_size = roi_size)
# feature_selector.set_filter('DF', dij=(1, 0), c = 0.1)
# feature_selector.set_filter('HARRIS', alpha = 0.04)
# feature_selector.set_filter('TRIGGS', alpha = 0.05)
# feature_selector.set_filter('HM')

feature_selector.set_roi(roi_size)

feature_selector.apply_filter()

- Set a picker and parse parameters
- Pick features. If no score_image is generated yet, pick_points() will call apply_filter() to generate one. pick_points() will also take a score_image directly, like this: pick_points(score_image).

In [ ]:
%matplotlib inline
feature_selector.set_picker('LM', min_distance = 10, n_points = 50)
# feature_selector.set_picker('ANMS', n_points=50)
# feature_selector.set_picker('DS', n_points=50, min_distance = 7, min_score = 50)
points = feature_selector.pick_points()

fig, ax = plt.subplots(2)
ax[0].imshow(reference_image, cmap = 'grey')
ax[0].plot(points[:,1], points[:,0], 'x')
ax[1].imshow(feature_selector.score_image, cmap = 'grey')
ax[1].plot(points[:,1], points[:,0], 'x')
plt.show()

### Masking
I had been using ROIselect (now SubsetSelection) to generate a ROI polygon. I only select features inside of the ROI

I think will think about how to include this more elegantly in the pipeline, but here is an example below

In [ ]:
# from pyidi.GUIs.selection import SubsetSelection
from matplotlib.path import Path

# roi_select = SubsetSelection(video)
# roi_select.polygon
# polygon_new = np.array(roi_select.polygon)

polygon = np.array([[4, 4, 97, 98, 4], [25, 218, 219, 25, 25]])
path = Path(polygon.T)


mask_image = path.contains_points(np.array([(i,j) for i in range(reference_image.shape[0]) for j in range(reference_image.shape[1])])).reshape(reference_image.shape)
mask_image = mask_image.reshape(reference_image.shape)

score_image = feature_selector.score_image
score_image[~mask_image] = 0

points = feature_selector.pick_points()

fig, ax = plt.subplots(2)
ax[0].imshow(reference_image, cmap = 'grey')
ax[0].plot(points[:,1], points[:,0], 'x')
ax[1].imshow(score_image, cmap = 'grey')
ax[1].plot(points[:,1], points[:,0], 'x')
plt.show()